SVMwoEdge.ipynb by:

Lloyd Fernandes (lloydf2@illinois.edu)

Project director: Richard Sowers r-sowers@illinois.eduhttps://publish.illinois.edu/r-sowers/ Copyright 2019 University of Illinois Board of Trustees. All Rights Reserved. Licensed under the MIT license

In [1]:
import pandas as pd
import numpy as np
from data import *
from feature_eng import *

In [2]:
df = pd.read_pickle("block4_1_features.pkl") #read the concatenated feature file
#df = df.droplevel('traj')
df.head()

lat        lon    speed  lon_acc  lat_acc type  \
id traj time                                                         
1  0    0.00  37.981384  23.734019  26.3600   0.0625  -0.0121  Bus   
        0.04  37.981386  23.734021  26.3634  -0.0149  -0.0141  Bus   
        0.08  37.981387  23.734024  26.3578  -0.0625  -0.0160  Bus   
        0.12  37.981389  23.734027  26.3454  -0.1101  -0.0179  Bus   
        0.16  37.981390  23.734030  26.3229  -0.2023  -0.0199  Bus   

              traveled_d              edge_id  avg_speed   bearing  \
id traj time                                                         
1  0    0.00      156.25  250699612_250709162   5.227568  0.169584   
        0.04      156.25  250699612_250709162   5.227568  0.474542   
        0.08      156.25  250699612_250709162   5.227568  0.251416   
        0.12      156.25  250699612_250709162   5.227568  0.474556   
        0.16      156.25  250699612_250709162   5.227568  0.169594   

              edge_progress  dir  edge_progress_intervals  vehicle_density  \
id traj time                                                                 
1  0    0.00       0.934900    0                      0.9                1   
        0.04       0.932464    0                      0.9                1   
        0.08       0.932163    0                      0.9                1   
        0.12       0.930224    0                      0.9                1   
        0.16       0.929970    0                      0.9                1   

              avg_surr_speed  xtrack_dist  
id traj time                               
1  0    0.00         26.3600    14.497745  
        0.04         26.3634    14.229915  
        0.08         26.3578    13.948674  
        0.12         26.3454    13.607180  
        0.16         26.3229    13.325939

In [3]:
#removing data from intersections.
def remove_intersection_data(df, edge_progress_min, edge_progress_max):
    df = df[(edge_progress_min <= df['edge_progress']) & (df['edge_progress'] <= edge_progress_max)]
    return df

df = remove_intersection_data(df,0.1,0.9)

In [4]:
#remove all vehicles except taxi and car
df = df[(df['type'] == 'Car' )|( df['type'] == 'Taxi')]


In [5]:
#prep labels
df_label =df.reset_index()[['id','edge_id','type']].drop_duplicates()
df_label.set_index(['id','edge_id'],inplace = True)
df_label['label'] = (df_label["type"] == 'Car').astype(int)

#add edge_id  as index
df.set_index('edge_id', append=True,inplace = True)
df = df.reorder_levels([0,1,3,2])
df.head()

lat        lon  speed  lon_acc  \
id traj edge_id             time                                         
3  0    250699612_250699613 0.00  37.981945  23.733895    0.0      0.0   
                            0.04  37.981945  23.733895    0.0      0.0   
                            0.08  37.981945  23.733895    0.0      0.0   
                            0.12  37.981945  23.733895    0.0      0.0   
                            0.16  37.981945  23.733895    0.0      0.0   

                                  lat_acc type  traveled_d  avg_speed  \
id traj edge_id             time                                        
3  0    250699612_250699613 0.00      0.0  Car        86.7   1.098968   
                            0.04      0.0  Car        86.7   1.098968   
                            0.08      0.0  Car        86.7   1.098968   
                            0.12      0.0  Car        86.7   1.098968   
                            0.16      0.0  Car        86.7   1.098968   

                                  bearing  edge_progress  dir  \
id traj edge_id             time                                
3  0    250699612_250699613 0.00      0.0        0.52701    0   
                            0.04      0.0        0.52701    0   
                            0.08      0.0        0.52701    0   
                            0.12      0.0        0.52701    0   
                            0.16      0.0        0.52701    0   

                                  edge_progress_intervals  vehicle_density  \
id traj edge_id             time                                             
3  0    250699612_250699613 0.00                      0.5                1   
                            0.04                      0.5                1   
                            0.08                      0.5                1   
                            0.12                      0.5                1   
                            0.16                      0.5                1   

                                  avg_surr_speed  xtrack_dist  
id traj edge_id             time                               
3  0    250699612_250699613 0.00             0.0    -1.141464  
                            0.04             0.0    -1.141464  
                            0.08             0.0    -1.141464  
                            0.12             0.0    -1.141464  
                            0.16             0.0    -1.141464

In [172]:
#aggregate function
df_agg = df.groupby(['id','traj','edge_id']).agg({

    'speed': ['mean', 'median', 'std'],
    'lon_acc': ['mean','median', 'std'],
    'lat_acc': ['mean','median', 'std'],
    'bearing':['mean','median','std'],
    'dir':['mean'],
    'vehicle_density':['mean','median','std'],
    'avg_surr_speed':['mean','median','std']
    'xtrack_dist' :['mean','median','std']
    
})
df_agg.head()




avg_surr_speed xtrack_dist
                                      mean         std
id traj edge_id                                       
3  0    250699612_250699613       0.497101    0.465291
   1    250699612_250699613       1.101357    0.610986
   2    250699612_250699613       0.758235    0.006123
4  0    250699613_250699614       4.515849    0.299160
   1    250699613_250699614       4.033802    0.554226

In [173]:
#join features
df_features = df_agg.join(df_label[['type','label']])
df_features.head()

C:\Users\lloyd\anaconda3\envs\ox\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


(avg_surr_speed, mean)  (xtrack_dist, std) type  \
id edge_id             traj                                                    
3  250699612_250699613 0                   0.497101            0.465291  Car   
                       1                   1.101357            0.610986  Car   
                       2                   0.758235            0.006123  Car   
4  250699613_250699614 0                   4.515849            0.299160  Car   
                       1                   4.033802            0.554226  Car   

                             label  
id edge_id             traj         
3  250699612_250699613 0         1  
                       1         1  
                       2         1  
4  250699613_250699614 0         1  
                       1         1

In [174]:
#merge id and traj
df_features.index = [df_features.index.get_level_values(1),df_features.index.map('{0[2]}/{0[0]}'.format)]
df_features.index.set_names("traj",inplace = True,level = 1)
df_features.head()

(avg_surr_speed, mean)  (xtrack_dist, std) type  \
edge_id             traj                                                    
250699612_250699613 0/3                 0.497101            0.465291  Car   
                    1/3                 1.101357            0.610986  Car   
                    2/3                 0.758235            0.006123  Car   
250699613_250699614 0/4                 4.515849            0.299160  Car   
                    1/4                 4.033802            0.554226  Car   

                          label  
edge_id             traj         
250699612_250699613 0/3       1  
                    1/3       1  
                    2/3       1  
250699613_250699614 0/4       1  
                    1/4       1

In [175]:
#edges which have both car and taxi
df_label_1 = df_label.reset_index()[["edge_id","type","label"]].drop_duplicates()
both_car_taxi = df_label_1.groupby(['edge_id']).agg({
   'label':'count'    
})
both_car_taxi = both_car_taxi[both_car_taxi['label'] >1]
both_car_taxi.index

Index(['250691795_250699359', '250698926_250699612', '250699359_250699613',
       '250699359_8446047162', '250699362_250699984', '250699362_250700083',
       '250699362_250708641', '250699362_599116897', '250699612_250699613',
       '250699613_250699614', '250699613_599116897', '250699614_250699615',
       '250699614_250699711', '250699614_8446047162', '250699711_250699973',
       '250699711_250708641', '250699711_599116897', '250699972_8446047162',
       '250700083_250700084', '250700083_388172056', '250700084_250700085',
       '250700084_388172051', '250700339_250706959', '250700339_364104830',
       '250706958_250708641', '250706958_364104825', '250706958_388172056',
       '250706958_388172075', '250708522_364104825', '250708556_250708641',
       '300400247_300400248', '300400248_8446047162', '388172051_388172055',
       '388172075_4464395371'],
      dtype='object', name='edge_id')

In [176]:
#get those entries which have both cars and taxis plying in the edge
df_features = df_features.loc[both_car_taxi.index]
df_features.head()

(avg_surr_speed, mean)  (xtrack_dist, std) type  \
edge_id             traj                                                     
250691795_250699359 0/88                23.169200            0.097230  Car   
                    0/102               27.976434            0.182141  Car   
                    0/106               21.666746            0.518709  Car   
                    1/128               10.049026            0.962530  Car   
                    2/128               11.458721            0.000000  Car   

                           label  
edge_id             traj          
250691795_250699359 0/88       1  
                    0/102      1  
                    0/106      1  
                    1/128      1  
                    2/128      1

In [177]:
#count of how many taxis and cars ply in an edge
#count_list = df_features.groupby('edge_id').count()['type']
count_list = df_features.reset_index('edge_id').pivot_table(index = 'edge_id',values = 'label',columns = 'type',aggfunc = ['count'])

In [178]:
#creating a mask to get equal numbers of cars and taxis in test set
df_mask = df_features.reset_index().set_index(['edge_id','traj','type']).groupby(['edge_id','type']).cumcount().to_frame()+1
df_mask = df_mask.join(count_list,on=['edge_id'])
test_limit = 0.9
df_mask["min_vehicle"] = [df_mask.iloc[i,2] if df_mask.iloc[i,1]>df_mask.iloc[i,2] else df_mask.iloc[i,1] for i in range(0,df_mask.shape[0])]

C:\Users\lloyd\anaconda3\envs\ox\lib\site-packages\pandas\core\reshape\merge.py:648: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [179]:
df_test_mask =df_mask[df_mask[0]>np.floor(test_limit*df_mask['min_vehicle'])]
df_test_mask = pd.MultiIndex.from_tuples(list(zip(df_test_mask.index.get_level_values(0),df_test_mask.index.get_level_values(1))),names = ['edge_id','traj'])
df_train_mask = df_mask[df_mask[0]<=np.floor(test_limit*df_mask['min_vehicle'])]
df_train_mask = pd.MultiIndex.from_tuples(list(zip(df_train_mask.index.get_level_values(0),df_train_mask.index.get_level_values(1))),names = ['edge_id','traj'])

In [180]:
#generate test set
df_test = df_features[df_features.index.isin(df_test_mask)]
df_test.shape

(758, 4)

In [181]:
#generate training set
df_train = df_features[df_features.index.isin(df_train_mask)]
df_train.shape

(622, 4)

In [182]:
df_train = df_train.dropna()
df_train_x = df_train.copy()
df_train_x.drop(['type','label'],axis = 1,inplace = True)
df_train_y = df_train['label']

In [183]:
#SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(kernel =  'rbf',gamma = 5e-7,C = 9))
clf.fit(df_train_x, df_train_y)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(C=9, gamma=5e-07))])

In [184]:
df_test = df_test.dropna()
df_test_x = df_test.copy()
df_test_x.drop(['type','label'],axis = 1,inplace = True)
df_test_y = df_test['label']

In [185]:
#SVC SCORE
clf.score(df_test_x,df_test_y)

0.5997357992073976

In [186]:
#naive prediction predict everything as 1
(df_test_y == 1).astype(int).mean()

0.9260237780713342

In [187]:
np.mean(np.array(clf.predict(df_test_x)==df_test_y).astype(int))

0.5997357992073976

In [44]:
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression(max_iter = 100000,random_state=0).fit(df_train_x, df_train_y)  
clf_log.score(df_test_x,df_test_y)

0.3817701453104359

In [47]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)

# Train the model using the training sets
model.fit(df_train_x,df_train_y)

KNeighborsClassifier(n_neighbors=3)

In [48]:
model.score(df_test_x,df_test_y)

0.43989431968295906